In [1]:
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import json
import pinecone
from dotenv import load_dotenv
import os


c:\Users\anees\Desktop\Coding\llmrepo\train-LLAMA\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
load_dotenv()

In [3]:
pc = Pinecone(api_key=os.getenv("PINECONE_KEY"))

In [4]:
with open("data.json", "r") as file:
    data = json.load(file)


In [5]:
def collect_data(data, key_filter=None, prefix=""):
    collected_data = []

    if isinstance(data, dict): 
        for key, value in data.items():
            new_prefix = f"{prefix}.{key}" if prefix else key 
            if key_filter is None or key in key_filter:
                if isinstance(value, str):  
                    collected_data.append((new_prefix, value))
                elif isinstance(value, (dict, list)): 
                    collected_data.extend(collect_data(value, key_filter, new_prefix))
            elif isinstance(value, (dict, list)):
                collected_data.extend(collect_data(value, key_filter, new_prefix))
    elif isinstance(data, list):  
        for i, item in enumerate(data):
            new_prefix = f"{prefix}[{i}]"  
            collected_data.extend(collect_data(item, key_filter, new_prefix))
    elif isinstance(data, str): 
        collected_data.append((prefix, data))

    return collected_data
collected_data = collect_data(data)


In [6]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [29]:
texts = [item[1] for item in collected_data]
texts

['A dedicated Electronics and Telecommunication professional with a strong background in machine learning, web development, and cloud platforms. Proven ability to lead projects, solve complex problems, and continuously learn new technologies to drive innovation and efficiency.',
 'Aneesh Patne',
 'aneeshpatne12@gmail.com',
 'https://www.linkedin.com/in/aneeshpatne',
 'https://github.com/aneeshpatne',
 'https://leetcode.com/aneeshpatne',
 'M.Tech in Electronics and Telecommunication',
 'Veermata Jijabai Technological Institute',
 'Mumbai, Maharashtra',
 '2023 - 2025',
 'Specialized in Machine Learning and Signal Processing. Relevant coursework includes Advanced Algorithms, Neural Networks, and Communication Systems.',
 'B.Tech in Electronics and Telecommunication',
 'Thakur College of Engineering and Technology',
 'Mumbai, Maharashtra',
 '2019 - 2023',
 'Graduated with First Class Honors. Engaged in multiple projects focusing on embedded systems and wireless communication.',
 'Python',


In [30]:
embeddings = model.encode(texts)

In [31]:
embedded_data = [
    {"prefix": collected_data[i][0], "text": texts[i], "embedding": embeddings[i].tolist()}
    for i in range(len(texts))
]

Save to PineCode

In [34]:
load_dotenv()

True

In [37]:
pc = Pinecone(
    api_key=os.getenv("PINECONE_KEY")
)

In [38]:
index_name = "rag-resume-data"
embedding_dim = 384
metric = "cosine"
if index_name not in pc.list_indexes().names():
    # Create the index
    pc.create_index(
        name=index_name,
        dimension=embedding_dim,
        metric=metric,
        spec=ServerlessSpec(
            cloud="aws",  # Choose your cloud provider
            region="us-east-1"  # Choose your region
        )
    )
print(f"Index '{index_name}' is ready.")


Index 'rag-resume-data' is ready.


In [7]:
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 384,
              'host': 'rag-resume-data-icsq7e1.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'rag-resume-data',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [9]:
indexes = pc.list_indexes()

In [ ]:
print

In [44]:
data_to_upsert = [
    {
        "id": f"doc-{i}",  
        "values": item["embedding"], 
        "metadata": {  
            "prefix": item["prefix"], 
            "text": item["text"] 
        }
    }
    for i, item in enumerate(embedded_data)
]

In [45]:
index.upsert(data_to_upsert)

{'upserted_count': 69}